In [1]:
import os
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score, precision_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import numpy as np


In [3]:

# Function to load images and labels from folder
def load_images_and_labels(folder, target_size=(224, 224)):
    images = []
    labels = []
    for subfolder in os.listdir(folder):
        subfolder_path = os.path.join(folder, subfolder)
        label = subfolder
        for filename in os.listdir(subfolder_path):
            image_path = os.path.join(subfolder_path, filename)
            try:
                img = image.load_img(image_path, target_size=target_size)
                img_array = image.img_to_array(img)
                img_array = preprocess_input(img_array)
                images.append(img_array)
                labels.append(label)
            except Exception as e:
                print(f"Error loading image: {image_path} - {str(e)}")
    return np.array(images), np.array(labels)


In [21]:
# Folder paths
train_folder = 'Train/train'
test_folder ='Test/test'
val_folder ='Val'

In [19]:

# Load images and labels for training, testing, and validation
X_train, y_train = load_images_and_labels(train_folder)
X_test, y_test = load_images_and_labels(test_folder)
X_val, y_val = load_images_and_labels(val_folder)


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Val/val'

In [23]:

# Load images and labels for training, testing, and validation
#X_train, y_train = load_images_and_labels(train_folder)
#X_test, y_test = load_images_and_labels(test_folder)
X_val, y_val = load_images_and_labels(val_folder)


In [25]:
# Load MobileNet model
base_model = MobileNet(weights='imagenet', include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(len(set(y_train)), activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)


C:\Users\KIIT\AppData\Local\Temp\ipykernel_33052\2865956719.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False)


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [27]:
from sklearn.preprocessing import LabelEncoder


# Convert string labels to integer labels
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)
y_val = label_encoder.transform(y_val)

# Freeze all layers in the base model
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val))


Epoch 1/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 212s 1s/step - accuracy: 0.7946 - loss: 0.6873 - val_accuracy: 0.9178 - val_loss: 0.2378
Epoch 2/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 111s 550ms/step - accuracy: 0.9397 - loss: 0.1853 - val_accuracy: 0.9091 - val_loss: 0.2714
Epoch 3/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 110s 549ms/step - accuracy: 0.9506 - loss: 0.1582 - val_accuracy: 0.9552 - val_loss: 0.1720
Epoch 4/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 112s 556ms/step - accuracy: 0.9738 - loss: 0.0858 - val_accuracy: 0.9539 - val_loss: 0.1571
Epoch 5/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 106s 530ms/step - accuracy: 0.9794 - loss: 0.0685 - val_accuracy: 0.9440 - val_loss: 0.1886
Epoch 6/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 109s 541ms/step - accuracy: 0.9836 - loss: 0.0527 - val_accuracy: 0.9639 - val_loss: 0.1431
Epoch 7/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 107s 533ms/step - accuracy: 0.9938 - loss: 0.0242 - val_accuracy: 0.9340 - val_loss: 0.2563
Epoch 8/10
201/201 ━━━━━━━━━━━━━━━━━━━━ 106s 529ms/step - accuracy: 0.9897 - lo

In [28]:

# Evaluate the model
train_pred_probs = model.predict(X_train)
train_pred = np.argmax(train_pred_probs, axis=1)
train_accuracy = accuracy_score(y_train, train_pred)
train_precision = precision_score(y_train, train_pred, average='weighted')
train_f1 = f1_score(y_train, train_pred, average='weighted')
train_roc_auc = roc_auc_score(y_train, train_pred_probs, multi_class='ovr')

val_pred_probs = model.predict(X_val)
val_pred = np.argmax(val_pred_probs, axis=1)
val_accuracy = accuracy_score(y_val, val_pred)
val_precision = precision_score(y_val, val_pred, average='weighted')
val_f1 = f1_score(y_val, val_pred, average='weighted')
val_roc_auc = roc_auc_score(y_val, val_pred_probs, multi_class='ovr')

test_pred_probs = model.predict(X_test)
test_pred = np.argmax(test_pred_probs, axis=1)
test_accuracy = accuracy_score(y_test, test_pred)
test_precision = precision_score(y_test, test_pred, average='weighted')
test_f1 = f1_score(y_test, test_pred, average='weighted')
test_roc_auc = roc_auc_score(y_test, test_pred_probs, multi_class='ovr')


201/201 ━━━━━━━━━━━━━━━━━━━━ 101s 490ms/step
26/26 ━━━━━━━━━━━━━━━━━━━━ 12s 456ms/step
25/25 ━━━━━━━━━━━━━━━━━━━━ 12s 474ms/step


In [31]:
from sklearn.metrics import recall_score

train_recall = recall_score(y_train, train_pred, average='weighted')
val_recall = recall_score(y_val, val_pred, average='weighted')
test_recall = recall_score(y_test, test_pred, average='weighted')


In [33]:

# Print evaluation metrics
print("Train data Accuracy:", train_accuracy)
print("Train data Precision:", train_precision)
print("Train data Recall:", train_recall)
print("Train data F1 Score:", train_f1)
print("Train data ROC AUC Score:", train_roc_auc)

print("\nValidation data Accuracy:", val_accuracy)
print("Validation data Precision:", val_precision)
print("Validation data Recall:", val_recall)
print("Validation data F1 Score:", val_f1)
print("Validation data ROC AUC Score:", val_roc_auc)

print("\nTest data Accuracy:", test_accuracy)
print("Test data Precision:", test_precision)
print("Test data Recall:", test_recall)
print("Test data F1 Score:", test_f1)
print("Test data ROC AUC Score:", test_roc_auc)



Train data Accuracy: 0.9940643548890972
Train data Precision: 0.9941319127098539
Train data Recall: 0.9940643548890972
Train data F1 Score: 0.994051603669099
Train data ROC AUC Score: 0.9999882557874604

Validation data Accuracy: 0.9489414694894147
Validation data Precision: 0.9491498111866932
Validation data Recall: 0.9489414694894147
Validation data F1 Score: 0.9486521840418539
Validation data ROC AUC Score: 0.9932962953780378

Test data Accuracy: 0.95
Test data Precision: 0.9502438323020723
Test data Recall: 0.95
Test data F1 Score: 0.9497818505183802
Test data ROC AUC Score: 0.9917708333333334


In [35]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def calculate_metrics(y_true, y_pred, y_prob):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_prob)
    return accuracy, precision, recall, f1, roc_auc

def print_metrics_table(train_metrics, val_metrics, test_metrics):
    print("Metrics\t\t\tTrain\t\tValidation\tTest")
    print("Accuracy\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[0], val_metrics[0], test_metrics[0]))
    print("Precision\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[1], val_metrics[1], test_metrics[1]))
    print("Recall\t\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[2], val_metrics[2], test_metrics[2]))
    print("F1 Score\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[3], val_metrics[3], test_metrics[3]))
    print("ROC AUC Score\t\t{:.4f}\t\t{:.4f}\t\t{:.4f}".format(train_metrics[4], val_metrics[4], test_metrics[4]))

# Assuming train_metrics, val_metrics, and test_metrics are tuples/lists containing metrics in the order: (accuracy, precision, recall, f1_score, roc_auc_score)
train_metrics = (train_accuracy, train_precision, train_recall, train_f1, train_roc_auc)
val_metrics = (val_accuracy, val_precision, val_recall, val_f1, val_roc_auc)
test_metrics = (test_accuracy, test_precision, test_recall, test_f1, test_roc_auc)

print_metrics_table(train_metrics, val_metrics, test_metrics)


Metrics			Train		Validation	Test
Accuracy		0.9941		0.9489		0.9500
Precision		0.9941		0.9491		0.9502
Recall			0.9941		0.9489		0.9500
F1 Score		0.9941		0.9487		0.9498
ROC AUC Score		1.0000		0.9933		0.9918


In [37]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/leaf miner/leaf miner90_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/leaf miner/leaf miner90_.jpg'

In [14]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/leaf miner/leaf miner1014_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted class: leaf miner
Confidence score: 0.973034


In [18]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/red rust/red rust960_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted class: red rust
Confidence score: 0.99999964


In [19]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/red rust/red rust1618_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step
Predicted class: red rust
Confidence score: 1.0


In [21]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/healthy/healthy111_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted class: healthy
Confidence score: 0.9999894


In [22]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/healthy/healthy1266_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicted class: healthy
Confidence score: 0.7758966


In [23]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/anthracnose/anthracnose100_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
Predicted class: anthracnose
Confidence score: 0.9999999


In [25]:
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet import preprocess_input
import numpy as np


# Define class names based on the unique labels found in your dataset
class_names = sorted(os.listdir(train_folder))

def predict_image_category(image_path, model, class_names):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    preds = model.predict(img_array)
    predicted_class = np.argmax(preds)
    confidence_score = preds[0][predicted_class]
    predicted_class_name = class_names[predicted_class]
    return predicted_class_name, confidence_score

# Example usage:
image_path = 'C:/Users/KIIT/Downloads/Cashew/Cashew/test/anthracnose/anthracnose1560_.jpg'
predicted_class_name, confidence_score = predict_image_category(image_path, model, class_names)
print("Predicted class:", predicted_class_name)
print("Confidence score:", confidence_score)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted class: anthracnose
Confidence score: 0.9999995


In [39]:
from tensorflow.keras.models import save_model
model.save('MobileNet.h5')